In [ ]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt

def preprocess(data, split_date):
    column_mapping = {
        'date': 'ds',
        'orders': 'y',
    }
    data.rename(columns=column_mapping, inplace=True)
    data['ds'] = pd.to_datetime(data['ds'])
    #print(data.head())

    train_data = data[data['ds'] < split_date]  # Дані до вказаної дати
    actual_data = data[data['ds'] >= split_date]   # Дані після вказаної дати
    return train_data, actual_data

def predict(train_data, periods):
    model = Prophet()
    model.fit(train_data)

    future = model.make_future_dataframe(periods=28, include_history=False)
    forecast = model.predict(future)
    model.plot_components(forecast)
    return forecast

def plot(train_data, forecast, comparison_df, yhat, periods):
    # Візуалізація прогнозу та фактичних даних
    plt.figure(figsize=(10, 6))
    plt.plot(train_data['ds'].head(periods), train_data['y'].head(periods), label='Тренувальні дані')
    plt.plot(forecast['ds'], forecast[yhat], label='Прогноз', linestyle='--')
    plt.plot(comparison_df['ds'], comparison_df['y'], label='Фактичні дані', linestyle=':', marker='o')
    plt.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'], color='gray', alpha=0.2)
    plt.legend()
    plt.xlabel('Дата')
    plt.ylabel('Цільова змінна')
    plt.title('Прогноз Prophet та фактичні дані')
    plt.show()

In [ ]:
file_path = 'files/orders_till_2024-11-10.csv'
split_date = '2024-11-01'
predict_periods = 28

data = pd.read_csv(file_path, usecols=['date', 'orders'])
train_data, actual_data = preprocess(data, split_date)
forecast = predict(train_data, predict_periods)

yhat = 'yhat'
comparison_df = forecast[['ds', yhat]].merge(actual_data, on='ds', how='left')
#print(comparison_df.tail(20))

plot(train_data, forecast, comparison_df, yhat, predict_periods)